In [2]:
import numpy as np
import pandas as pd  
import matplotlib.pylab as plt 
from matplotlib import pyplot as plt1
import seaborn as sns 
%matplotlib inline  

# read the datafile using panda library.  ensure right file location on machine. 
data = pd.read_csv(r"C:\Users\SAARTH CHAHAL\Desktop\Programming\AIML\smoking_driking_dataset_Ver01.csv")
# EDA (Exploratory Data Analysis): 
# Determine number of rows and colums in the provided data
data.shape 
data.head()
data.nunique(axis=0)
data_cleaned = data.dropna(axis=0)
data_cleaned = data_cleaned[data_cleaned['waistline'].between(25,150)]   
# sight_left above 5 is based on observation of the data  
data_cleaned = data_cleaned[data_cleaned['sight_left'] < 5 ]
# sight_left above 5 is based on observation of the data  
data_cleaned = data_cleaned[data_cleaned['sight_right'] < 5 ]
#since in correlation down the line we will require all number we will need to drop sex which takes string as input.  
data_cleaned = data_cleaned.drop('sex',axis=1) 
# convert drinker as Y or N 
data_cleaned['DRK_YN'] = np.where(data_cleaned['DRK_YN'] == 'Y', 1,0 ) 
dfdata= pd.DataFrame(data_cleaned)  


# Learning model : Decison Tree ->  Random Forest. 

from sklearn.model_selection import train_test_split 
# Train  model  
# Data  consist of key health parameters in X1 array  that contains the features to train on, 
# And a y1 array(SMK_stat_type_cd) with the target variable, 
X1=dfdata[['tot_chole','HDL_chole','LDL_chole','triglyceride','hemoglobin',
           'urine_protein','serum_creatinine','SGOT_AST','SGOT_ALT','gamma_GTP',
           'waistline','age','SBP','DBP','BLDS','height','weight','sight_left','sight_right']]
y1=dfdata['SMK_stat_type_cd']
# Data consist of key health parameters inarray that contains the features to train on, 
# And a y2 array(DRK_YN)
X2=dfdata[['tot_chole','HDL_chole','LDL_chole','triglyceride','hemoglobin',
           'urine_protein','serum_creatinine','SGOT_AST','SGOT_ALT','gamma_GTP',
           'waistline','age','SBP','DBP','BLDS','height','weight','sight_left','sight_right']]
y2=dfdata['DRK_YN']


# Train test split. test split is 30 % train set is 70 % 
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3)

## Loading the Decison Tree .    
# Start with training a single decision tree.   X1 set for Smokers and X2 set for Drikers 

from sklearn.tree import DecisionTreeClassifier 

dtree1 = DecisionTreeClassifier() 
dtree1.fit(X1_train,y1_train) 

dtree2 = DecisionTreeClassifier() 
dtree2.fit(X2_train,y2_train) 

# Start evaluating the decison tree and prediction on Training data  
# Predict class or regression value for X.
# For a classification model, the predicted class for each sample in X is returned.
# For a regression model, the predicted value based on X is returned.

predictions1 = dtree1.predict(X1_test)
predictions2 = dtree2.predict(X2_test)

from sklearn.metrics import classification_report,confusion_matrix 

# Confusion matrix to evaluate the accuracy of a classification. 
# Classfication Report. Builds a text report showing the main classification metrics 

print(classification_report(y1_test,predictions1)) 
print(confusion_matrix(y1_test,predictions1)) 

print(classification_report(y2_test,predictions2)) 
print(confusion_matrix(y2_test,predictions2)) 



## compare the decision tree model to a random forest. 

from sklearn.ensemble import RandomForestClassifier

# A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples 
# of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.
# The sub-sample size is controlled with the max_samples parameter if bootstrap=True (default), 
#  otherwise the whole dataset is used to build each tree.

rfc1 = RandomForestClassifier(n_estimators=100)
rfc2 = RandomForestClassifier(n_estimators=100)

rfc1.fit(X1_train, y1_train)
rfc1_pred = rfc1.predict(X1_test) 

rfc2.fit(X2_train, y2_train)
rfc2_pred = rfc2.predict(X2_test) 

print(classification_report(y1_test,rfc1_pred))
print(confusion_matrix(y1_test,rfc1_pred))

print(classification_report(y2_test,rfc2_pred))
print(confusion_matrix(y2_test,rfc2_pred))



              precision    recall  f1-score   support

           0       0.63      0.63      0.63    147705
           1       0.63      0.63      0.63    147958

    accuracy                           0.63    295663
   macro avg       0.63      0.63      0.63    295663
weighted avg       0.63      0.63      0.63    295663

[[93021 54684]
 [54020 93938]]
              precision    recall  f1-score   support

           0       0.72      0.72      0.72    147705
           1       0.72      0.72      0.72    147958

    accuracy                           0.72    295663
   macro avg       0.72      0.72      0.72    295663
weighted avg       0.72      0.72      0.72    295663

[[106993  40712]
 [ 41075 106883]]
